# Ensebles

Un ensemble è una collezione di classificatori, tutti addestrati per performare lo stesso compito.

Un ensemble può consistere in diverse versioni dello stesso modello, o di modelli diversi.

L'output finale per un classificatore ensemble è tipicamente ottenere attraverso una media (pesata) delle predictions dei singoli modelli dell'ensemble.

Un ensemble di diversi modelli che conseguono una simile performance generalizzativa, spesso superano in performance i risultati dei singoli modelli.

---

# Ensemble intuition

Supponiamo di avere un ensemble di funzioni di classificatori binari $f_k(x)$ per $k = 1,...,K$.

Supponiamo inoltre che mediamente abbiano lo stesso errore di classificazione $\epsilon = E_{p(x,y)}[y \neq f_k(x)] < 0.5$, ma che gli errori che commettono siano indipendenti.

L'intuizione sta nel fatto che la maggioranza dei $K$ classificatori, dovrebbe classificare correttamente un dato esempio, dove un singolo classificatore potrebbe fallire.

Un semplice voto di maggioranza potrebbe migliorare la performance di classificazione diminuendo la varianza.

---

# Proof of Majority Voting

Supponiamo di avere un ensemble di $K$ classificatori binari, dove ogni classificatore ha una accuracy $\alpha \gt 0.5$.

La maggioranza dei votanti commette un errore quando $\frac{K}{2} + 1$ classificatori sbagliano.

L'errore segue la distribuzione binomiale:

#### $P(x \leq k) = \sum_{i=0}^{k} \binom{K}{k} \alpha^i (1-\alpha)^{K-i}$

---

## Esempio

Si suppone di avere 5 classificatori con accuracy $\alpha = 0.75$. L'errore di probabilità dell'ensemble è: (l'ensemble sbaglia quando almeno 3 classificatori sbaglianom da questo $x \leq 2$)

### $P(x \leq 2) = \sum_{i=0}^{2} \binom{5}{i} 0.75^i (0.25)^{5-i} = 0.105$

L'accuratezza dell'ensemble è $1-P(x \leq 2) = 0.895$.

<span style="color:gold;">Questo è vero quando gli errori tra i classificatori non sono correlati</span>

---

# Independent Training Sets

Supponiamo di collezionare diversi training set indipendenti $T_{r1}, ..., T_{rK}$ e di usarli per addestrare diverse istanze dello stesso classificatore ottenendo $K$ funzioni di classificazione $f_1(x), ..., f_K(x)$. Quindi traning set indipendenti si traduce dati diversi che descrivono lo stesso problema.

I classificatori addestrati in questo modo sono garantiti di commettere errori indipendenti sui dati di test.

Se l'errore atteso di ogni classificatore è minore di 0.5, allora il voto di maggioranza pesato è garantito di ridurre l'errore di generalizzazione atteso.

<span style="color:gold;">Qual è il punto debole di questo approccio?</span>
- Se devo costruire diversi training set che sono indipendenti l'un l'altro e descrivono lo stesso problema sto sicuramente perdendo informazioni. Inoltre non ho la garanzia che allenando i classificatori su questi training set indipendenti, questi performino bene nel mondo reale.

---

# Bagging

Bootstrap aggregation o <span style="color:gold;">Bagging</span> è un'approssimazione al metodo precedente che prende un singolo training set $T_r$ e ne prende K sottocampioni casuali (con rimpiazzo) per formare K training set $T_{r1}, ..., T_{rK}$.

Ogni training set viene usato per addestrare una diversa istanza dello stesso classificatore ottenendo $K$ funzioni di classificazione $f_1(x), ..., f_K(x)$.

Si gioca sul fatto che prendendo un campione casuale, la probabilità di avere lo stesso esempio su ogni dataset è molto bassa.

Con questo metodo uso tutto il dataset e ogni classificatore ha la stessa probabilità di vedere ogni esempio.

Come contro si ha che gli errori non saranno totalmente indipendenti perchè i training set non sono indipendenti, ma il campionamento casuale di solito introduce abbastanza diversità per diminuire la varianza e dare una performance migliore.

![Ensemble_Adaboost1](./images/Ensemble_Adaboost1.png)

Può succedere che un elemento sia presente più di una volta nello stesso dataset. <- non ci sono garanzie da questo punto di vista

---

# Bagging proof

Supponiamo che ogni classificatore impari una funzione $y = f_i(x) + e_i$ dove $e$ è l'errore.

L'errore medio di $M$ classificatori sul dataset $D$ è:

### $Ɛ_{AV} = \frac{1}{M} \sum_{i=1}^{M} E_D[e_i]$

Dove $E_D[e_i]$ è l'expected value sul dataset $D$ dell'errore $e_i$.

La predizione bagged è:

### $y_{bagged} = \frac{1}{M} \sum_{i=1}^{M} f_i(x) + e_i(x)$

Dove $f_i(x) + e_i(x)$ è la combinazione delle risposte di ogni singolo classificatore

L'errore medio del classificatore bagged è:

### $Ɛ = E_D[\frac{1}{M} \sum_{i=1}^{M} e_i(x)]$

Che è l'expected value sul world dataset del valore medio di $e$ per il singolo elemento $x$

La differenza tra $Ɛ_{AV} = \frac{1}{M} \sum_{i=1}^{M} E_D[e_i]$, e $Ɛ = E_D[\frac{1}{M} \sum_{i=1}^{M} e_i(x)]$ è che il primo prendendo $E_d[E_i]$ è il "negato" dell'accuracy del classificatore (ovvero 1-accuracy), mentre il secondo, ogni errore conta per ogniclassificatore che ho considerato, mentre ogni errore nel primo viene contato come 1.<br>
Quindi tipicamente se gli errori non sono correlati:

$Ɛ = E_D[\frac{1}{M} \sum_{i=1}^{M} e_i(x)]$ $\leq$ $Ɛ_{AV} = \frac{1}{M} \sum_{i=1}^{M} E_D[e_i]$

È uguale solo nel caso in cui tutti i classificatori commettono lo stesso errore, altrimenti nel secondo caso sto contando un errore come $\frac{1}{\frac{1}{M}}$, mentre nel primo caso solo come $\frac{1}{M}$.

---

# Boosting

L'idea è quella di combinare i classificatori non in parallelo ma in serie.

Vogliamo che un classificatore che fa parte della serie di classificatore sia bravo a risolvere solo una porzione del problema.

L'idea è quella di sfruttare una catena dove ogni classificatore sa qualcosa riguardo i classificatori precedenti. Inoltre un classificatore nella catena, conosce anche gli elementi nei quali il classificatore precedente commette una predizione errata.

![Ensemble_Adaboost2](./images/Ensemble_Adaboost2.png)

Ogni volta che qualche canale sbaglia a predirre il meteo, il suo valore di fiducia diminuisce.

---

# Algoritmo Boosting

Il boosting parte da un dataset $D$ e addestra sequenzialmente classificatori uguali $f_i$ focalizzandosi sugli errori dei classificatori precedenti.

Assegna ad ogni elemento del mio dataset $D$ $x_k $ pesi uguali $w_k = \frac{1}{N}$

Itera con $i$ da 1 a $M$ numero classificatori che ho nella catena:
1. Prendi un nuovo dataset $D_i$ da $D$ usando i pesi $\{w_i^k\}^N_{k=1}$ come probabilità di sampling per ogni record $x$
2. Addestra il classificatore $f_i$ su $D_i$ e misura l'accuratezza e registrala come $\alpha_i$
3. Se $x_k$ è stato classificato erroneamente aumenta il suo peso al prossimo stage $w_{i+1}$ e ricalcola i pesi

La regola di decisione finale è $y(x_{new}) = sign(\sum_{i}^{M} \alpha_i f_i(x_{new}))$

Ovvero il segno  (positivo o negativo) della somma del prodotto tra la fiducia e la risposta del singolo classificatore

![Ensemble_Adaboost3](./images/Ensemble_Adaboost3.png)

Il boosting allena sequenzialmente i classificatori aumentando l'importanza degli elementi classificati erroneamente.

Ogni classificatore nella catena deve avere un'accuratezza maggiore di 0.5.

Agli elementi classificati non correttamente viene fornito un peso maggiore ed è più probabile che siano campionati pian piano che la catena aumenta.

Catene più lunghe tendono ad aumentare l'accuratezza ma anche il rischio di overfitting.

---

# Sampling from a Discrete Distribution Example

Un possibile modo per campionare una distribuzione discreta è usando l'algoritmo <span style="color:gold;">cdf</span>.

***Input***: una distribuzione discreta $H(x)$ con $x \in \{1, ..., N\}$ con $N$ numero di bin e $K$ numero di campioni da generare.

***Output***: $K$ indici.

***Step 1***: costruire la ***cdf table*** $T$:

$T_x = [\sum_0^{x-1} H(x), \sum_0^x (x)H(x)]$

Che è una matrice con due elementi per riga, dove ci sono tante righe quanti gli elementi nel dataset. Il primo numero è la somma dell'istogramma di tutti gli elementi precedenti, mentre il secondo è la somma dell'istogramma di tutti gli elementi precedenti più l'istogramma dell'elemento corrente.

***Step 2***: Draw indexes:

```
while num iter = K do:<br>
    pesco un numero casuale $n \in [0,1]$<br>
    controllo in quale intervallo è contenuto $n$ -> find $k | T_k[0] \leq n \lt T_k[1]$<br>
    output $k$
end
```

![Ensemble_Adaboost4](./images/Ensemble_Adaboost4.png)

Il campionamento può essere molto lento perché devo aggiornare la table ogni volta!

---



# Adaboost

Per evitare di campionare ogni volta, si può usare un algoritmo che non campiona, ma che aggiorna i pesi.

Posso rimuovere il campionamento sfruttando il fatto che ogni elemento è fornito di un peso, che aumenta nel caso sia classificato erroneamente e diminuisce nel caso sia stato classificato correttamente. Il peso ci fornisce una misura di quanto l'elemento sia preciso o meno. Se un elemento ha un peso molto alto, avrà un impatto maggiore nella misura della accuracy. 

Quindi senza dover campionare, posso comunque prendere in considerazione i pesi, e posso campionare la ***weighted accuracy function***. In questo caso sto computando la ***weighted error function***:

#### L'errore del classificatore allo stato $k$ $\epsilon_k = \sum_{i=1}^N w^i_k \mathbb I (f_k(x^i)) \neq y^i$

Ciclando su tutto il dataset sommo i pesi dell'elemento corrente che contano se l'elemento è stato preditto nel modo giusto o sbagliato

Dove $\mathbb I$ è una **indicator function** che mappa tutto ciò che è classificato erroneamente a 1 e tutto ciò che è classificato correttamente a 0.

Se l'elemento è stato predetto erroneamente la funzione indicatrice varrà 1 e quindi si aggiungerà alla somma totale il peso degli elementi.

Quindi se l'elemento pesa di più l'errore è più importante, e se pesa di meno è meno importante.

![Ensemble_Adaboost5](./images/Ensemble_Adaboost5.png)

- All'inizio gli elementi hanno lo stesso peso, $\frac{1}{N}$
- Alleno il classificatore sull'intero dataset $D$
- Ma il mio obiettivo è quello di minimizzare l'errore pesato $\epsilon_k$
- Immagazzino la fiducia del classificatore $\alpha_k = \log(\frac{1-\epsilon_{ki}}{\epsilon_k})$ &rarr; più è alta la fiducia, più un errore aumenterà il peso di tanto e viceversa
<br>
- Dopo aver allenato i classificatori, aggiorno i pesi degli elementi usando la logica dell'algoritmo di boosting
  - se l'elemento è stato classificato correttamente, diminuisco il suo peso &rarr; $w_{k+1} = w_k e^{-\alpha_k}$
  - se l'elemento è stato classificato erroneamente, aumento il suo peso &rarr; $w_{k+1} = w_k e^{\alpha_k}$

---

# Adaboost Re-weighting

- Dopo ogni stage dell'algoritmo di boosting, i pesi degli elementi vengono cambiati

- Gli elementi classificati erroneamente hanno il loro peso aumentato $w_{i_{k+1}} = w_{i_k} e^{\alpha_k}$

- Gli elementi classificati correttamente hanno il loro peso diminuito $w_{i_{k+1}} = w_{i_k} e^{-\alpha_k}$

- Questo significa che ad ogni stage gli elementi più classificati erroneamente sono più importanti per il classificatore corrente

- Ogni classificatore in ogni stage deve ancora raggiungere un'accuratezza maggiore di 0.5, ma in termini di weighted accuracy

- Condizione necessaria per ogni stage del boosting
    ### $(1 - \epsilon_i) > 0.5$ 

    Che è la versione pesata dell'accuracy ($1 - \epsilon_i$ è la weighted accuracy)

- altrimenti la catena viene riavviata

---


# Adaboost Classifier trust and decision rule

- Ogni classificatore nella catena ha una fiducia associata o <span style="color:gold;">Classifier Trust</span>
- La Classifier Trust è la quantità di fiducia che la decisione finale dovrebbe mettere in un classificatore specifico della catena
- Impatta la regola di decisione in cui i classificatori più accurati sono più fidati

La fiducia per lo stage $k$ è 
    $\alpha_k = log(\frac{1-\epsilon_k}{\epsilon_k})$

La regola di decisione finale è una somma pesata delle risposte individuali
    $y^i = sign(\sum_{k=1}^M \alpha_k f_k(x_i))$

---

# Weak Classifier

Un classificatore debole è un classificatore semplice con alcuni elementi stocastici al suo interno che gli permette di essere riavviato (?).

Per costruire un classificatore debole per $D_j$ ripeti finchè $\epsilon_j \lt 0.5$:
- scegli casualmente una dimensione dei dati
- scegli casualmente una soglia
- imposta tutti i punti sopra la soglia (lungo l'asse scelto) a 1 e tutti i punti sotto a -1 (o viceversa)
- calcola $\epsilon_j$ su $D_j$

Ripeti finché l'accuracy è maggiore di 0.5

---

Il boosting permette di ottenere da una catena di classificatori lineari, un classificatore "complessivo" 
non lineare.